# 第六模块 (1)：评测数据集生成

本 Notebook 用于生成 RAG 系统的评测数据集：

### 用户与意图细分

**用户画像**：
1. 家庭亲子出游
2. 商务出差人士
3. 朋友结伴旅行
4. 情侣浪漫度假
5. 独自旅行探索

**意图方向**：
- 设施类
  - 房间设施
  - 公共设施
  - 餐饮设施
- 服务类
  - 前台服务
  - 客房服务
  - 退房/入住效率
- 位置类
  - 交通便利性
  - 周边配套
  - 景观/朝向
- 价格类
  - 性价比
  - 价格合理性
- 体验类
  - 整体满意度
  - 安静程度
  - 卫生状况

### 数据集构建策略

评测数据集包含 100 个问题：
1. **结构化问题（70个）**：用户画像 × 意图方向 = 5 × 14
2. **模糊性问题（10个）**：开放性、模糊性、复合意图
3. **时效性问题（10个）**：5 个明确时效性 + 5 个隐含时效性
4. **特定房型问题（10个）**：4 个模糊房型 + 6 个精确房型

总计：70 + 10 + 10 + 10 = 100 个问题

## 环境配置

In [1]:
import os
import json
import time
import random
import pandas as pd
from pathlib import Path
from tqdm.notebook import tqdm
from lib import LLMClient

In [2]:
# 检查环境变量
api_key = os.getenv("DASHSCOPE_API_KEY")
if not api_key:
    raise EnvironmentError("缺少环境变量: DASHSCOPE_API_KEY")

print(f"API Key: {api_key[:10]}...{api_key[-4:]}")

API Key: sk-5540c66...f37a


In [3]:
# 项目路径配置
PROJECT_ROOT = Path.cwd()
DATA_DIR = PROJECT_ROOT / "data"
EVAL_DATA_DIR = DATA_DIR / "evaluation"
EVAL_DATA_DIR.mkdir(parents=True, exist_ok=True)

# 初始化客户端
LLM_MODEL = "deepseek-v3.2"
llm_client = LLMClient(api_key=api_key, model=LLM_MODEL, json=False)

print(f"项目根目录: {PROJECT_ROOT}")
print(f"评测数据目录: {EVAL_DATA_DIR}")
print(f"LLM 模型: {LLM_MODEL}")

项目根目录: C:\Users\22418\Desktop\Scorpio\复旦\课程\大三下\酒店评论
评测数据目录: C:\Users\22418\Desktop\Scorpio\复旦\课程\大三下\酒店评论\data\evaluation
LLM 模型: deepseek-v3.2


## 定义配置

In [4]:
# 用户画像定义
USER_PERSONAS = ["家庭亲子", "商务出差", "朋友出游", "情侣出游", "独自旅行"]

# 意图方向定义
INTENT_DIRECTIONS = [
    "房间设施", "公共设施", "餐饮设施",
    "前台服务", "客房服务", "退房/入住效率",
    "交通便利性", "周边配套", "景观/朝向",
    "价格合理性", "性价比",
    "整体满意度", "安静程度", "卫生状况"
]

print(f"用户画像数量: {len(USER_PERSONAS)}")
print(f"意图方向数量: {len(INTENT_DIRECTIONS)}")
print(f"结构化问题数量: {len(USER_PERSONAS) * len(INTENT_DIRECTIONS)}")

用户画像数量: 5
意图方向数量: 14
结构化问题数量: 70


In [5]:
# 房型定义（已剔除评论总数 < 50 条的房型）
EXACT_ROOM_TYPES = ['花园大床房', '花园双床房', '城央绿意大床房', '城央绿意双床房', '红棉大床套房', '红棉双床套房']
FUZZY_ROOM_TYPES = ['大床房', '双床房', '套房', '主题房']

print(f"精确房型数量: {len(EXACT_ROOM_TYPES)}")
print(f"模糊房型数量: {len(FUZZY_ROOM_TYPES)}")

精确房型数量: 6
模糊房型数量: 4


## 数据集生成器

In [6]:
class EvaluationDatasetGenerator:
    """评测数据集生成器"""
    
    def __init__(self, llm_client, user_personas: list[str], intent_directions: list[str], exact_room_types: list[str], fuzzy_room_types: list[str]):
        self.llm_client = llm_client
        self.user_personas = user_personas
        self.intent_directions = intent_directions
        self.exact_room_types = exact_room_types
        self.fuzzy_room_types = fuzzy_room_types
    
    def generate_structured_question(self, persona: str, intent: str, question_id: int) -> dict:
        """生成结构化问题（用户画像 × 意图方向）"""
        prompt = f"""
你是一个酒店预订平台的用户，正在咨询广州花园酒店相关信息。

【用户画像】
{persona}

【关注方向】
{intent}

【任务】
请根据上述用户画像和关注方向，生成1个真实、自然的酒店咨询问题。

【要求】
1. 问题要符合该用户画像的特点和需求
2. 问题要聚焦在指定的关注方向上
3. 问题要口语化、自然，像真实用户会问的
4. 问题长度控制在20字以内
5. 不要在问题中直接提及酒店名称、用户画像或关注方向的名称
6. 关注从酒店评论中可能获取到答案的问题，不要提问需要调取其他工具才能获得答案的问题（如当前客房余量、床的具体尺寸等）

【输出格式】
只输出问题文本，不要有任何额外说明或标记。

示例：
- 酒店有没有儿童游乐设施？适合带小孩去吗？
- 房间里的床大吗？睡得舒服吗？
- 离地铁站近吗？交通方不方便？
"""
        
        question_text = self.llm_client.generate(prompt)
        question_text = question_text.strip()
        
        return {
            "question_id": question_id,
            "question_type": "structured",
            "question": question_text,
            "metadata": {
                "persona": persona,
                "intent_direction": intent
            }
        }
    
    def generate_ambiguous_questions(self, num_questions: int, start_id: int) -> list[dict]:
        """生成模糊性问题（开放性、复合意图）"""
        prompt = f"""
你是一个酒店预订平台的用户，正在咨询广州花园酒店相关信息。

【任务】
请生成{num_questions}个模糊性、开放性或复合意图的酒店咨询问题。

【问题特点】
1. **模糊性**：问题表述不够明确，可能有多种理解
2. **开放性**：问题范围较广，没有聚焦在单一方面
3. **复合意图**：一个问题中包含多个关注点

【要求】
1. 问题要口语化、自然，像真实用户会问的
2. 问题长度控制在10-30字
3. 每个问题独立一行
4. 不要在问题中直接提及酒店名称，不要编号，不要任何额外说明
5. 关注从酒店评论中可能获取到答案的问题，不要提问需要调取其他工具才能获得答案的问题（如当前客房余量、床的具体尺寸等）

【示例】
这个酒店怎么样？
酒店整体感觉如何？有什么特色？
想带家人去，酒店各方面条件都还行吧？
交通方便吗？周边有什么好玩的？吃饭方便不？
"""
        
        response = self.llm_client.generate(prompt)
        questions = [q.strip() for q in response.strip().split('\n') if q.strip()]
        
        results = []
        for i, question_text in enumerate(questions[:num_questions]):
            results.append({
                "question_id": start_id + i,
                "question_type": "ambiguous",
                "question": question_text,
                "metadata": {}
            })
        
        return results

    def generate_temporal_questions(self, num_clear: int, num_implied: int, start_id: int) -> list[dict]:
        """生成时效性问题"""
        results = []
        
        # 生成明确时效性问题
        prompt_clear = f"""
你是一个酒店预订平台的用户，正在咨询广州花园酒店相关信息，特别关注最新、最近的情况。

【任务】
请生成{num_clear}个具有**明确时效性**的酒店咨询问题。

【明确时效性要求】
问题中必须包含以下词汇之一：
- "最近"、"最新"、"近期"、"这段时间"
- "现在"、"目前"、"当前"
- "今年"、"这个月"

【要求】
1. 问题要口语化、自然，像真实用户会问的
2. 问题长度控制在20字以内
3. 每个问题独立一行
4. 不要在问题中直接提及酒店名称，不要编号，不要任何额外说明
5. 关注从酒店评论中可能获取到答案的问题，不要提问需要调取其他工具才能获得答案的问题（如当前客房余量、床的具体尺寸等）

【示例】
酒店最近的服务质量怎么样？
这段时间酒店卫生状况如何？
近期酒店有什么新的服务或设施？
"""
        
        response_clear = self.llm_client.generate(prompt_clear)
        questions_clear = [q.strip() for q in response_clear.strip().split('\n') if q.strip()]
        
        for i, question_text in enumerate(questions_clear[:num_clear]):
            results.append({
                "question_id": start_id + i,
                "question_type": "temporal_clear",
                "question": question_text,
                "metadata": {
                    "time_sensitivity": "clear"
                }
            })
        
        # 生成隐含时效性问题
        prompt_implied = f"""
你是一个酒店预订平台的用户，正在咨询广州花园酒店相关信息，隐含关注时效性但表述不明确。

【任务】
请生成{num_implied}个具有**隐含时效性**的酒店咨询问题。

【隐含时效性特点】
1. 不直接使用"最近"、"最新"等明确时间词
2. 但隐含关注当前状态或变化趋势
3. 可能使用"还"、"仍然"、"依然"等词暗示时间对比

【要求】
1. 问题要口语化、自然，像真实用户会问的
2. 问题长度控制在20字以内
3. 每个问题独立一行
4. 不要在问题中直接提及酒店名称，不要编号，不要任何额外说明
5. 关注从酒店评论中可能获取到答案的问题，不要提问需要调取其他工具才能获得答案的问题（如当前客房余量、床的具体尺寸等）

【示例】
酒店的早餐还是自助餐吗？
听说装修过了，现在房间怎么样？
酒店的服务水平保持得如何？
"""
        
        response_implied = self.llm_client.generate(prompt_implied)
        questions_implied = [q.strip() for q in response_implied.strip().split('\n') if q.strip()]
        
        for i, question_text in enumerate(questions_implied[:num_implied]):
            results.append({
                "question_id": start_id + num_clear + i,
                "question_type": "temporal_implied",
                "question": question_text,
                "metadata": {
                    "time_sensitivity": "implied"
                }
            })
        
        return results
    
    def generate_room_type_questions(self, num_fuzzy: int, num_exact: int, start_id: int) -> list[dict]:
        """生成特定房型问题"""
        results = []
        
        # 生成模糊房型问题
        sampled_fuzzy = random.sample(self.fuzzy_room_types, min(num_fuzzy, len(self.fuzzy_room_types)))
        for i, room_type in enumerate(sampled_fuzzy):
            prompt = f"""
你是一个酒店预订平台的用户，正在咨询广州花园酒店的"{room_type}"相关信息。

【任务】
请生成1个关于"{room_type}"的真实、自然的咨询问题。

【要求】
1. 问题必须明确提到"{room_type}"
2. 问题要关注该房型的具体特点（设施、环境、价格等），不要关注对所有房型答案都一样的问题
3. 问题要口语化、自然，像真实用户会问的
4. 问题长度控制在20字以内
5. 不要在问题中直接提及酒店名称
6. 关注从酒店评论中可能获取到答案的问题，不要提问需要调取其他工具才能获得答案的问题（如当前客房余量、床的具体尺寸等）

【输出格式】
只输出问题文本，不要有任何额外说明。

示例：
- 大床房的床有多大？适合两个人睡吗？
- 双床房房间空间大吗？
"""
            
            question_text = self.llm_client.generate(prompt)
            results.append({
                "question_id": start_id + i,
                "question_type": "room_type_fuzzy",
                "question": question_text.strip(),
                "metadata": {
                    "room_type": room_type,
                    "room_type_category": "fuzzy"
                }
            })
        
        # 生成精确房型问题
        sampled_exact = random.sample(self.exact_room_types, min(num_exact, len(self.exact_room_types)))
        for i, room_type in enumerate(sampled_exact):
            prompt = f"""
你是一个酒店预订平台的用户，正在咨询广州花园酒店的"{room_type}"相关信息。

【任务】
请生成1个关于"{room_type}"的真实、自然的咨询问题。

【要求】
1. 问题必须明确提到"{room_type}"
2. 问题要关注该房型的具体特点（设施、环境、价格等），不要关注对所有房型答案都一样的问题
3. 问题要口语化、自然，像真实用户会问的
4. 问题长度控制在20字以内
5. 不要在问题中直接提及酒店名称
6. 关注从酒店评论中可能获取到答案的问题，不要提问需要调取其他工具才能获得答案的问题（如当前客房余量、床的具体尺寸等）

【输出格式】
只输出问题文本，不要有任何额外说明。

示例：
- 花园大床房的景观怎么样？能看到什么风景？
- 红棉大床套房面积多大？适合带孩子吗？
"""
            
            question_text = self.llm_client.generate(prompt)
            results.append({
                "question_id": start_id + num_fuzzy + i,
                "question_type": "room_type_exact",
                "question": question_text.strip(),
                "metadata": {
                    "room_type": room_type,
                    "room_type_category": "exact"
                }
            })
        
        return results
    
    def generate_complete_dataset(self) -> list[dict]:
        """生成完整的评测数据集"""
        all_questions = []
        question_id = 1
        
        # 1. 生成结构化问题
        structured_count = 0
        for persona in tqdm(self.user_personas, desc="用户画像"):
            for intent in self.intent_directions:
                try:
                    question = self.generate_structured_question(persona, intent, question_id)
                    all_questions.append(question)
                    question_id += 1
                    structured_count += 1
                    time.sleep(0.5)
                
                except Exception as e:
                    print(f"\n结构化问题 [{persona} × {intent}] 生成失败: {str(e)}")
        
        print(f"结构化问题生成完成: {structured_count} 个")
        
        time.sleep(1)
        
        # 2. 生成模糊性问题
        try:
            ambiguous_questions = self.generate_ambiguous_questions(
                num_questions=10,
                start_id=question_id
            )
            all_questions.extend(ambiguous_questions)
            question_id += len(ambiguous_questions)
            print(f"模糊性问题生成完成: {len(ambiguous_questions)} 个")
        
        except Exception as e:
            print(f"模糊性问题生成失败: {str(e)}")
        
        time.sleep(1)
        
        # 3. 生成时效性问题
        try:
            temporal_questions = self.generate_temporal_questions(
                num_clear=5,
                num_implied=5,
                start_id=question_id
            )
            all_questions.extend(temporal_questions)
            question_id += len(temporal_questions)
            print(f"时效性问题生成完成: {len(temporal_questions)} 个")
        
        except Exception as e:
            print(f"时效性问题生成失败: {str(e)}")
        
        time.sleep(1)
        
        # 4. 生成特定房型问题
        try:
            room_type_questions = self.generate_room_type_questions(
                num_fuzzy=4,
                num_exact=6,
                start_id=question_id
            )
            all_questions.extend(room_type_questions)
            print(f"特定房型问题生成完成: {len(room_type_questions)} 个")
        
        except Exception as e:
            print(f"特定房型问题生成失败: {str(e)}")

        print(f"数据集生成完成, 问题总数: {len(all_questions)} 个")
        
        return all_questions

In [7]:
# 初始化生成器
generator = EvaluationDatasetGenerator(
    llm_client=llm_client,
    user_personas=USER_PERSONAS,
    intent_directions=INTENT_DIRECTIONS,
    exact_room_types=EXACT_ROOM_TYPES,
    fuzzy_room_types=FUZZY_ROOM_TYPES
)

## 构建评测数据集

In [8]:
# 生成评测数据集
questions = generator.generate_complete_dataset()

用户画像:   0%|          | 0/5 [00:00<?, ?it/s]

结构化问题生成完成: 70 个
模糊性问题生成完成: 10 个
时效性问题生成完成: 10 个
特定房型问题生成完成: 10 个
数据集生成完成, 问题总数: 100 个


In [9]:
# 保存评测数据集
eval_file = EVAL_DATA_DIR / "eval_set.json"

with open(eval_file, 'w', encoding='utf-8') as f:
    json.dump(questions, f, ensure_ascii=False, indent=2)

print(f"评测数据集已保存: {eval_file}")

评测数据集已保存: C:\Users\22418\Desktop\Scorpio\复旦\课程\大三下\酒店评论\data\evaluation\eval_set.json
